In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)



e:\User\PC\ads_enefit_ueh\ads_enfit


In [2]:
from libs.common import *
from config.config import *

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


e:/User/PC/ads_enefit_ueh/ads_enfit


In [3]:
if os.path.exists(exps_dir) == False: # tạo thư mục (nếu chưa có)
  os.makedirs(exps_dir, exist_ok=True)

save_dir = f"{exps_dir}/exp2"
os.makedirs(save_dir, exist_ok=True)

# 1. Load data

In [4]:
## Define important columns
data_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id']
client_cols = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_prices_cols = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_prices_cols = ['forecast_date', 'euros_per_mwh']
forecast_weather_cols = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_weather_cols = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols = ['longitude', 'latitude', 'county']
target_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

root=data_dir
## Importing only specified columns
df_data = pl.read_csv(os.path.join(root, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client = pl.read_csv(os.path.join(root, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas_prices = pl.read_csv(os.path.join(root, "gas_prices.csv"), columns=gas_prices_cols, try_parse_dates=True)
df_electricity_prices = pl.read_csv(os.path.join(root, "electricity_prices.csv"), columns=electricity_prices_cols, try_parse_dates=True)
df_forecast_weather = pl.read_csv(os.path.join(root, "forecast_weather.csv"), columns=forecast_weather_cols, try_parse_dates=True)
df_historical_weather = pl.read_csv(os.path.join(root, "historical_weather.csv"), columns=historical_weather_cols, try_parse_dates=True)
df_weather_station_to_county_mapping = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_target = df_data.select(target_cols)

## Prepare schema for api submission
schema_data = df_data.schema
schema_client = df_client.schema
schema_gas  = df_gas_prices.schema
schema_electricity = df_electricity_prices.schema
schema_forecast = df_forecast_weather.schema
schema_historical = df_historical_weather.schema
schema_target = df_target.schema

# 2. Transform and pre-processing 

In [5]:
def generate_features(df_data, df_client, df_gas_prices, df_electricity_prices, df_forecast_weather, df_historical_weather, df_weather_station_to_county_mapping, df_target):
    df_data = (
        df_data.with_columns(pl.col("datetime").cast(pl.Date).alias("date"),)
    )
    
    df_gas_prices = (
        df_gas_prices.rename({"forecast_date": "date"})
    )
    
    df_electricity_prices = (
        df_electricity_prices.rename({"forecast_date": "datetime"})
    )
    
    df_weather_station_to_county_mapping = (
        df_weather_station_to_county_mapping.with_columns(pl.col("latitude").cast(pl.datatypes.Float32),pl.col("longitude").cast(pl.datatypes.Float32))
    )
    
    # sum of all product_type targets related to ["datetime", "county", "is_business", "is_consumption"]
    df_target_all_type_sum = (
        df_target.group_by(["datetime", "county", "is_business", "is_consumption"]).sum().drop("product_type")
    )
    
    df_forecast_weather = (
        df_forecast_weather.rename({"forecast_datetime": "datetime"}).filter(pl.col("hours_ahead") >= 24) # we don't need forecast for today
        .with_columns(pl.col("latitude").cast(pl.datatypes.Float32),pl.col("longitude").cast(pl.datatypes.Float32),
            # datetime for forecast in a different timezone
            pl.col('datetime').dt.replace_time_zone(None).cast(pl.Datetime("us")),
        )
        .join(df_weather_station_to_county_mapping, how="left", on=["longitude", "latitude"]).drop("longitude", "latitude")
    )
    
    df_historical_weather = (
        df_historical_weather
        .with_columns(pl.col("latitude").cast(pl.datatypes.Float32),pl.col("longitude").cast(pl.datatypes.Float32),
        )
        .join(df_weather_station_to_county_mapping, how="left", on=["longitude", "latitude"]).drop("longitude", "latitude")
    )
    
    # creating average forecast characteristics for all weather stations
    df_forecast_weather_date = (
        df_forecast_weather.group_by("datetime").mean().drop("county")
    )
    
    # creating average forecast characteristics for weather stations related to county
    df_forecast_weather_local = (
        df_forecast_weather.filter(pl.col("county").is_not_null()).group_by("county", "datetime").mean()
    )
    
    # creating average historical characteristics for all weather stations
    df_historical_weather_date = (
        df_historical_weather.group_by("datetime").mean().drop("county")
    )
    
    # creating average historical characteristics for weather stations related to county
    df_historical_weather_local = (
        df_historical_weather.filter(pl.col("county").is_not_null()).group_by("county", "datetime").mean()
    )
    
    df_data = (
        df_data
        # pl.duration(days=1) shifts datetime to join lag features (usually we join last available values)
        .join(df_gas_prices.with_columns((pl.col("date") + pl.duration(days=1)).cast(pl.Date)), on="date", how="left")
        .join(df_client.with_columns((pl.col("date") + pl.duration(days=2)).cast(pl.Date)), on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity_prices.with_columns(pl.col("datetime") + pl.duration(days=1)), on="datetime", how="left")
        
        # lag forecast_weather features (24 hours * days)
        .join(df_forecast_weather_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_weather_local, on=["county", "datetime"], how="left", suffix="_fl")
        .join(df_forecast_weather_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fd_7d")
        .join(df_forecast_weather_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_fl_7d")

        # lag historical_weather features (24 hours * days)
        .join(df_historical_weather_date.with_columns(pl.col("datetime") + pl.duration(days=2)), on="datetime", how="left", suffix="_hd_2d")
        .join(df_historical_weather_local.with_columns(pl.col("datetime") + pl.duration(days=2)), on=["county", "datetime"], how="left", suffix="_hl_2d")
        .join(df_historical_weather_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_hd_7d")
        .join(df_historical_weather_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_hl_7d")
        
        # lag target features (24 hours * days)
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        .join(df_target_all_type_sum.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "is_consumption", "datetime"], suffix="_all_type_sum", how="left")
        
        
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
            pl.col("datetime").dt.hour().alias("hour"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.year().alias("year"),
        )
        .with_columns(
            pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("segment"),
        )
        # cyclical features encoding https://towardsdatascience.com/cyclical-features-encoding-its-about-time-ce23581845ca
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
        
        .drop("datetime", "hour", "dayofyear")
    )
    
    return df_data

In [6]:
## Function defined for merging dataframes with the row_id as unique identifier
def to_pandas(X, y=None):
    cat_cols = ["county", "is_business", "product_type", "is_consumption", "segment"]
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    
    return df

In [14]:
df_data, y = df_data.drop("target"), df_data.select("target")

df_train_features = generate_features(df_data, df_client, df_gas_prices, df_electricity_prices, df_forecast_weather, df_historical_weather, df_weather_station_to_county_mapping, df_target)

df_train_features = to_pandas(df_train_features, y)
# a little proportion of target values are null
df_train_features = df_train_features[df_train_features['target'].notnull()]

# filter old data
df_train_features = df_train_features[df_train_features.year >= 2022]

df_train_features.head()

,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id,...,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target,target_mean,target_std,target_ratio
row_id,,,,,,,,,,,,,,,,,,,,,
366048,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_1_0,0.017166,0.999853,0.0,1.0,0.000,0.279667,0.569453,7.000000
366049,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_1_1,0.017166,0.999853,0.0,1.0,442.226,412.962311,22.043884,1.382337
366050,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_2_0,0.017166,0.999853,0.0,1.0,0.000,0.000000,0.000000,0.000000
366051,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_2_1,0.017166,0.999853,0.0,1.0,44.899,43.781834,2.328470,1.557458
366052,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_3_0,0.017166,0.999853,0.0,1.0,0.015,0.295667,0.274237,0.869186


In [15]:
df_train_features

,county,is_business,product_type,is_consumption,prediction_unit_id,date,lowest_price_per_mwh,highest_price_per_mwh,origin_date,data_block_id,...,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target,target_mean,target_std,target_ratio
row_id,,,,,,,,,,,,,,,,,,,,,
366048,0,0,1,0,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_1_0,0.017166,0.999853,0.000000,1.000000,0.000,0.279667,0.569453,7.000000
366049,0,0,1,1,0,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_1_1,0.017166,0.999853,0.000000,1.000000,442.226,412.962311,22.043884,1.382337
366050,0,0,2,0,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_2_0,0.017166,0.999853,0.000000,1.000000,0.000,0.000000,0.000000,0.000000
366051,0,0,2,1,1,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_2_1,0.017166,0.999853,0.000000,1.000000,44.899,43.781834,2.328470,1.557458
366052,0,0,3,0,2,2022-01-01,80.0,94.0,2021-12-30,122.0,...,2022,0_0_3_0,0.017166,0.999853,0.000000,1.000000,0.015,0.295667,0.274237,0.869186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,1,64,2023-05-31,29.0,34.0,2023-05-29,637.0,...,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,197.233,250.953674,106.506004,0.985350
2018348,15,1,1,0,59,2023-05-31,29.0,34.0,2023-05-29,637.0,...,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000,0.000000,0.000000,0.000000
2018349,15,1,1,1,59,2023-05-31,29.0,34.0,2023-05-29,637.0,...,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,28.404,34.342167,2.976434,0.942917


In [16]:
df_train_features.to_csv(f'{save_dir}/df_train_features.csv')

In [7]:
data_cols = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'prediction_datetime', 'row_id']

In [35]:
df_target.columns

['target',
 'county',
 'is_business',
 'product_type',
 'is_consumption',
 'datetime']

In [9]:
df_new_target.columns

['county',
 'is_business',
 'product_type',
 'target',
 'is_consumption',
 'datetime']

In [11]:
df_target=df_new_target[df_new_target.columns]
df_target

county,is_business,product_type,target,is_consumption,datetime
i64,i64,i64,f64,i64,datetime[μs]
0,0,1,2.675,0,2023-05-26 00:00:00
0,0,1,471.887,1,2023-05-26 00:00:00
0,0,2,0.0,0,2023-05-26 00:00:00
0,0,2,5.414,1,2023-05-26 00:00:00
0,0,3,13.899,0,2023-05-26 00:00:00
…,…,…,…,…,…
15,1,0,188.167,1,2023-05-29 23:00:00
15,1,1,0.0,0,2023-05-29 23:00:00
15,1,1,31.484,1,2023-05-29 23:00:00


In [20]:
root=f"{data_dir}\\example_test_files"
df_test = pl.read_csv(os.path.join(root, "test.csv"), columns=data_cols[1:], try_parse_dates=True)
df_test=df_test.rename({'prediction_datetime':'datetime'})
df_client = pl.read_csv(os.path.join(root, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas_prices = pl.read_csv(os.path.join(root, "gas_prices.csv"), columns=gas_prices_cols, try_parse_dates=True)
df_electricity_prices = pl.read_csv(os.path.join(root, "electricity_prices.csv"), columns=electricity_prices_cols, try_parse_dates=True)
df_new_forecast_weather = pl.read_csv(os.path.join(root, "forecast_weather.csv"), columns=forecast_weather_cols, try_parse_dates=True)
df_new_historical_weather = pl.read_csv(os.path.join(root, "historical_weather.csv"), columns=historical_weather_cols, try_parse_dates=True)
# df_weather_station_to_county_mapping = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_new_target = pl.read_csv(os.path.join(root, "revealed_targets.csv"), columns=target_cols, try_parse_dates=True)
df_forecast_weather = pl.concat([df_forecast_weather, df_new_forecast_weather]).unique(['forecast_datetime', 'latitude', 'longitude', 'hours_ahead'])
    
df_historical_weather = pl.concat([df_historical_weather, df_new_historical_weather]).unique(['datetime', 'latitude', 'longitude'])
df_target = pl.concat([df_target, df_new_target]).unique(['datetime', 'county', 'is_business', 'product_type', 'is_consumption'])

In [17]:
df_test

county,is_business,product_type,is_consumption,datetime,row_id
i64,i64,i64,i64,datetime[μs],i64
0,0,1,0,2023-05-28 00:00:00,2005872
0,0,1,1,2023-05-28 00:00:00,2005873
0,0,2,0,2023-05-28 00:00:00,2005874
0,0,2,1,2023-05-28 00:00:00,2005875
0,0,3,0,2023-05-28 00:00:00,2005876
…,…,…,…,…,…
15,1,0,1,2023-05-31 23:00:00,2018347
15,1,1,0,2023-05-31 23:00:00,2018348
15,1,1,1,2023-05-31 23:00:00,2018349


In [10]:
df_new_target

county,is_business,product_type,target,is_consumption,datetime
i64,i64,i64,f64,i64,datetime[μs]
0,0,1,2.675,0,2023-05-26 00:00:00
0,0,1,471.887,1,2023-05-26 00:00:00
0,0,2,0.0,0,2023-05-26 00:00:00
0,0,2,5.414,1,2023-05-26 00:00:00
0,0,3,13.899,0,2023-05-26 00:00:00
…,…,…,…,…,…
15,1,0,188.167,1,2023-05-29 23:00:00
15,1,1,0.0,0,2023-05-29 23:00:00
15,1,1,31.484,1,2023-05-29 23:00:00


In [18]:
df_test_features = generate_features(
        df_test, 
        df_client, 
        df_gas_prices, 
        df_electricity_prices, 
        df_forecast_weather, 
        df_historical_weather, 
        df_weather_station_to_county_mapping, 
        df_target
    )
df_test_features = to_pandas(df_test_features)
df_test_features = df_test_features.drop('date',axis=1)

In [19]:
df_test_features

,county,is_business,product_type,is_consumption,lowest_price_per_mwh,highest_price_per_mwh,eic_count,installed_capacity,euros_per_mwh,hours_ahead,...,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour),target_mean,target_std,target_ratio
row_id,,,,,,,,,,,,,,,,,,,,,
2005872,0,0,1,0,28.299999,34.099998,507,4960.214844,87.540001,46.0,...,5,2023,0_0_1_0,0.565345,-0.824855,0.000000,1.000000,2.675000,NaN,NaN
2005873,0,0,1,1,28.299999,34.099998,507,4960.214844,87.540001,46.0,...,5,2023,0_0_1_1,0.565345,-0.824855,0.000000,1.000000,471.886993,NaN,NaN
2005874,0,0,2,0,28.299999,34.099998,11,34.000000,87.540001,46.0,...,5,2023,0_0_2_0,0.565345,-0.824855,0.000000,1.000000,0.000000,NaN,NaN
2005875,0,0,2,1,28.299999,34.099998,11,34.000000,87.540001,46.0,...,5,2023,0_0_2_1,0.565345,-0.824855,0.000000,1.000000,5.414000,NaN,NaN
2005876,0,0,3,0,28.299999,34.099998,1516,15977.559570,87.540001,46.0,...,5,2023,0_0_3_0,0.565345,-0.824855,0.000000,1.000000,13.899000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,1,29.000000,34.000000,15,620.000000,-1.290000,45.0,...,5,2023,15_1_0_1,0.522133,-0.852864,-0.258819,0.965926,183.352264,7.132739,NaN
2018348,15,1,1,0,29.000000,34.000000,20,624.500000,-1.290000,45.0,...,5,2023,15_1_1_0,0.522133,-0.852864,-0.258819,0.965926,0.000000,0.000000,NaN
2018349,15,1,1,1,29.000000,34.000000,20,624.500000,-1.290000,45.0,...,5,2023,15_1_1_1,0.522133,-0.852864,-0.258819,0.965926,34.539001,3.143939,NaN


In [21]:
df_test_features.to_csv(f'{save_dir}/df_test_features.csv')